In [ ]:
%load_ext autoreload
%autoreload 2



In [17]:
DATASETS = ['wisconsin', 'bupa', 'bands', 'ionosphere', 'hepatitis', 'saheart',
            'haberman', 'housevotes', 'heart', 'australian', 'tic-tac-toe',
            'mammographic', 'monk-2', 'appendicitis', 'breast', 'crx']
random_shuffles = [42, 69, 22, 1, 99]
COMPLEXITY_MEASURES = ['f2', 't4', 'c1', 'n3', 'l2', 'density']
BASE_CLFS = ['knn', 'bayes', 'dt', 'svm']

In [16]:
from mlutils.mlflow.utils import create_runs_for_params, get_unfinished_run_ids, get_runs, get_unfinished_runs, get_run_params, experiment_name_to_id
from glob import glob
from mlutils.distributedcomputing.experiments import run_experiments
from sklearn.model_selection import ParameterGrid
from mlflow import MlflowClient
import mlflow
from pathlib import Path
from datasetz.core.load_dataset import load_embedded_dataset

client = MlflowClient(tracking_uri="http://192.168.1.181:5010")
mlflow.set_tracking_uri("http://192.168.1.181:5010")



In [19]:
params = ParameterGrid({
    'dataset': DATASETS,
    # 'base_clf': ["svm", "dt", "knn"],
    'data_shuffle_random_state': random_shuffles,
    'should_take_test': [True, False],
    'min_samples': [28],
    "complexity_measure": ["n3"],
    "min_split_percentage": [0.183],
    "n_estimators": [3,5,7],
})


print(f"params={len(params)}")

params=480


In [20]:
runs = create_runs_for_params(params, experiment_name='04.08-quadsplit-explain-v1-extended', client=client)

2024-08-04 13:43:32.706 | INFO     | mlutils.mlflow.utils:create_runs_for_params:32 - Experiment not found, creating new. Id=74
100%|██████████| 480/480 [00:29<00:00, 16.42it/s]


In [6]:
runs = create_runs_for_params(params, experiment_name='09.07-quadsplit-explain-v1', client=client)

2024-07-09 00:02:50.625 | INFO     | mlutils.mlflow.utils:create_runs_for_params:32 - Experiment not found, creating new. Id=60
100%|██████████| 960/960 [00:34<00:00, 28.02it/s]


In [9]:
runsv2 = create_runs_for_params(params, experiment_name='09.07-quadsplit-explain-v2', client=client)

2024-07-09 00:07:41.114 | INFO     | mlutils.mlflow.utils:create_runs_for_params:32 - Experiment not found, creating new. Id=61
100%|██████████| 2880/2880 [02:00<00:00, 23.83it/s]


In [9]:
experiment_name_to_id('06.07-quadsplit-3', client)

'58'

In [10]:
runs= get_runs(experiment_name_to_id('06.07-quadsplit-3', client))

In [21]:
run_experiments(runs, 'experiment_quad_split_explain', notebook_path="quadsplit_entrypoint-2.ipynb", output_dir_path=Path("/tmp"), backend='pueue', pueue_group="quad-split")


2024-08-04 13:44:10.733 | INFO     | mlutils.distributedcomputing.experiments:run_experiments:13 - {'notebook_path': 'quadsplit_entrypoint-2.ipynb', 'output_dir_path': PosixPath('/tmp'), 'pueue_group': 'quad-split'}
2024-08-04 13:44:10.734 | WARNING  | mlutils.distributedcomputing.experiments:run_experiments:91 - Using pueue backend. Make sure you execute this function not from experiment notebook!
2024-08-04 13:44:10.821 | INFO     | mlutils.distributedcomputing.job_scheduling:create_experiment_notebook:101 - 
    Creating experiment notebook
        notebook_name=quadsplit_entrypoint-2.ipynb
        experiment_function=experiment_quad_split_explain
        instance_id_param_name=EXPERIMENT_INSTANCE_ID
        
2024-08-04 13:44:10.826 | INFO     | mlutils.distributedcomputing.experiments:run_experiments:108 - Notebook path=/Users/bgulowaty/studia/projekty/quad-splits/quadsplit_entrypoint-2-9670d642-dd6e-4fd7-b7a0-5bece18c00d0.ipynb
2024-08-04 13:44:10.826 | INFO     | mlutils.distribu

New task added (id 1280).
New task added (id 1281).
New task added (id 1282).
New task added (id 1283).
New task added (id 1284).
New task added (id 1285).
New task added (id 1286).
New task added (id 1287).
New task added (id 1288).


2024-08-04 13:44:11.049 | DEBUG    | mlutils.distributedcomputing.job_scheduling:run_experiments_in_puepe:219 - /opt/homebrew/bin/pueue add -g quad-split "/Users/bgulowaty/studia/projekty/quad-splits/.venv/bin/papermill --log-output --log-level DEBUG /Users/bgulowaty/studia/projekty/quad-splits/quadsplit_entrypoint-2-9670d642-dd6e-4fd7-b7a0-5bece18c00d0.ipynb /tmp/a158fa3800c64e529487fd32fa394964.ipynb -p EXPERIMENT_INSTANCE_ID a158fa3800c64e529487fd32fa394964"
2024-08-04 13:44:11.073 | DEBUG    | mlutils.distributedcomputing.job_scheduling:run_experiments_in_puepe:219 - /opt/homebrew/bin/pueue add -g quad-split "/Users/bgulowaty/studia/projekty/quad-splits/.venv/bin/papermill --log-output --log-level DEBUG /Users/bgulowaty/studia/projekty/quad-splits/quadsplit_entrypoint-2-9670d642-dd6e-4fd7-b7a0-5bece18c00d0.ipynb /tmp/806d4e2c52a1466fb75068b3b4d21ad1.ipynb -p EXPERIMENT_INSTANCE_ID 806d4e2c52a1466fb75068b3b4d21ad1"
2024-08-04 13:44:11.230 | DEBUG    | mlutils.distributedcomputing.jo

New task added (id 1289).
New task added (id 1290).
New task added (id 1291).


2024-08-04 13:44:11.293 | DEBUG    | mlutils.distributedcomputing.job_scheduling:run_experiments_in_puepe:219 - /opt/homebrew/bin/pueue add -g quad-split "/Users/bgulowaty/studia/projekty/quad-splits/.venv/bin/papermill --log-output --log-level DEBUG /Users/bgulowaty/studia/projekty/quad-splits/quadsplit_entrypoint-2-9670d642-dd6e-4fd7-b7a0-5bece18c00d0.ipynb /tmp/eee75dab384844a391b9430f52f84e36.ipynb -p EXPERIMENT_INSTANCE_ID eee75dab384844a391b9430f52f84e36"
2024-08-04 13:44:11.322 | DEBUG    | mlutils.distributedcomputing.job_scheduling:run_experiments_in_puepe:219 - /opt/homebrew/bin/pueue add -g quad-split "/Users/bgulowaty/studia/projekty/quad-splits/.venv/bin/papermill --log-output --log-level DEBUG /Users/bgulowaty/studia/projekty/quad-splits/quadsplit_entrypoint-2-9670d642-dd6e-4fd7-b7a0-5bece18c00d0.ipynb /tmp/258ed8ea0aca455491ab5856e5f9f624.ipynb -p EXPERIMENT_INSTANCE_ID 258ed8ea0aca455491ab5856e5f9f624"
2024-08-04 13:44:11.350 | DEBUG    | mlutils.distributedcomputing.jo

...

New task added (id 1747).
New task added (id 1748).
New task added (id 1749).
New task added (id 1750).
New task added (id 1751).
New task added (id 1752).


2024-08-04 13:44:31.247 | DEBUG    | mlutils.distributedcomputing.job_scheduling:run_experiments_in_puepe:219 - /opt/homebrew/bin/pueue add -g quad-split "/Users/bgulowaty/studia/projekty/quad-splits/.venv/bin/papermill --log-output --log-level DEBUG /Users/bgulowaty/studia/projekty/quad-splits/quadsplit_entrypoint-2-9670d642-dd6e-4fd7-b7a0-5bece18c00d0.ipynb /tmp/4b3a4a048f934785a3ba7246876bfefa.ipynb -p EXPERIMENT_INSTANCE_ID 4b3a4a048f934785a3ba7246876bfefa"
2024-08-04 13:44:31.334 | DEBUG    | mlutils.distributedcomputing.job_scheduling:run_experiments_in_puepe:219 - /opt/homebrew/bin/pueue add -g quad-split "/Users/bgulowaty/studia/projekty/quad-splits/.venv/bin/papermill --log-output --log-level DEBUG /Users/bgulowaty/studia/projekty/quad-splits/quadsplit_entrypoint-2-9670d642-dd6e-4fd7-b7a0-5bece18c00d0.ipynb /tmp/92de66c0e82b49a4bc54976195bd6c22.ipynb -p EXPERIMENT_INSTANCE_ID 92de66c0e82b49a4bc54976195bd6c22"
2024-08-04 13:44:31.371 | DEBUG    | mlutils.distributedcomputing.jo

New task added (id 1753).
New task added (id 1754).
New task added (id 1755).
New task added (id 1756).
New task added (id 1757).


2024-08-04 13:44:31.461 | DEBUG    | mlutils.distributedcomputing.job_scheduling:run_experiments_in_puepe:219 - /opt/homebrew/bin/pueue add -g quad-split "/Users/bgulowaty/studia/projekty/quad-splits/.venv/bin/papermill --log-output --log-level DEBUG /Users/bgulowaty/studia/projekty/quad-splits/quadsplit_entrypoint-2-9670d642-dd6e-4fd7-b7a0-5bece18c00d0.ipynb /tmp/7fe72edb888a406a961a0ef87129d9da.ipynb -p EXPERIMENT_INSTANCE_ID 7fe72edb888a406a961a0ef87129d9da"


New task added (id 1758).
New task added (id 1759).
